# Leave One Out Analyses

This code was run in the UK Biobank Research Analysis Platform to create the LOO analyses, which were used to test whether any individual variants were causing excessive amounts of horizontal pleiotropy. The inputs are the Bx, Bxse, By, and Byse calculated in the Mendelian randomization analyses.

In [ ]:
# Create empty list to store beta results
BetasNo <- list()


# Creating different dataframes excluding different variants (thru excluding row i)
for(i in 1:97){
  
  # Creating list of data frames w/ elements
  BetasNo[[i]] <- Betas[-i , ]
  
  
}


# Create empty coeffVector vector to store coefficient estimates
coeffVector <- c()



# Repeat this for loop logic to create mrinputs
for(i in 1:97){
  
  Betas <- BetasNo[[i]]
  
  # Creating mr_input object
  mrinput <- mr_input(
    bx = Betas$Bx,
    bxse = Betas$Bxse,
    by = Betas$By,
    byse = Betas$Byse,
    exposure = "Yin",
    outcome = "Yang"
  )
  
  ivw <- mr_ivw(mrinput,
                model = "default",
                penalized = FALSE,
                robust = FALSE,
                weights = "simple",
                alpha = 0.05)
  
  
  coeff <- ivw@Estimate
  
  
  coeffVector[i] <- coeff
  
  
  
  
}

coeffVector <- as.data.frame(coeffVector)
colnames(coeffVector) <- "x"

mean(coeffVector$x)


# Converting units to those we used
coeffVector$x <- coeffVector$x*10000

summary(coeffVector$x)


hist(coeffVector$x, xlab = "Additive hazard increase from 1 SD Increase in WHRadjBMI", main = "Leave-One-Out Estimates for IVW Model (WHRadjBMI)",sub = "Males with Less than a Secondary Education", font.sub = 2)
abline(v=mean(coeffVector$x),col="black",lwd=2, lty = 2, xlim = c(0,25))
text(-32,20, "Mean of LOO Analyses", col = "black", srt = 90)
abline(v=-28.37, col = "blue", lty = 2, xlim = c(0,25))
text(-27, 20, "Original Point Estimate", col = "blue", srt = 90)
